In [54]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd

# Base URL
url = "https://www.yoox.com/us/men/shoponline/misbhv_d#"
url = "https://www.yoox.com/us/men/shoponline/jil%20sander_d#" 

headers = {
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; rv:91.0) Gecko/20100101 Firefox/91.0'
}

# Send a GET request to fetch the page content
response = requests.get(url, headers=headers, timeout=10)

if response.status_code == 200:
    print("response successfully retrieved")
else:
    print("failed to retrieve response")

response successfully retrieved


In [58]:
soup = BeautifulSoup(response.text, 'html.parser')
items = soup.find_all('div', class_='itemContainer')
df = pd.DataFrame(columns = ('link', 'brand', 'category', 'old_price', 'new_price', 'sizes'))

for item in items:
    item_data = item.find('div', class_='itemData text-center')

    link = item_data.find('a', class_='itemlink')['href'] if item_data.find('a', class_='itemlink') else 'NA'
    brand = item_data.find('div', class_='brand').text.strip() if item_data.find('div', class_='brand') else 'NA'
    category = item_data.find('div', class_='microcategory').text.strip() if item_data.find('div', class_='microcategory') else 'NA'
    
    
    old_price = float(item_data.find('span', class_='oldprice').text.replace("$", "").replace(",", "").strip()) if item_data.find('span', class_='oldprice') else 'NA'
    new_price = float(item_data.find('div', class_='retail-newprice').text.replace("$", "").replace(",", "").strip()) if item_data.find('div', class_='retail-newprice') else 'NA'
    per_off = new_price/old_price if (new_price != 'NA' and old_price != 'NA') else 0
    
    sizes_elements = item_data.find_all('span', class_='aSize')
    sizes = ', '.join([size.text for size in sizes_elements]) if sizes_elements else 'NA'

    new_row = {'link' : f'https://www.yoox.com{link}',
               'brand' : brand,
               'category' : category,
               'sizes' : sizes,
               'old_price' : old_price,
               'new_price' : new_price,
               'per_off' : per_off}
        
    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

df.sort_values(by='per_off', ascending=False)
#df

,link,brand,category,old_price,new_price,sizes,per_off
40,https://www.yoox.com/us/17731105PX/item#dept=m...,JIL SANDER,Laced shoes,566.0,426.0,9,0.752650
32,https://www.yoox.com/us/17768001TB/item#dept=m...,JIL SANDER,Sandals,614.0,462.0,"7, 8, 9, 10, 11",0.752443
68,https://www.yoox.com/us/17751393DN/item#dept=m...,JIL SANDER,Sandals,614.0,462.0,"7, 8, 9, 10, 11, 12",0.752443
36,https://www.yoox.com/us/17741809OO/item#dept=m...,JIL SANDER,Sneakers,709.0,533.0,8,0.751763
60,https://www.yoox.com/us/17796588AG/item#dept=m...,JIL SANDER,Boots,1196.0,899.0,"6, 7, 8, 9, 10",0.751672
...,...,...,...,...,...,...,...
17,https://www.yoox.com/us/30156973DK/item#dept=m...,JIL SANDER,Casual pants,943.0,268.0,36,0.284199
116,https://www.yoox.com/us/14372004UM/item#dept=m...,JIL SANDER,Sweaters,1681.0,450.0,"36, 40, 42",0.267698
15,https://www.yoox.com/us/30159687IP/item#dept=m...,JIL SANDER,Denim pants,754.0,185.0,"30, 32",0.245358
19,https://www.yoox.com/us/30124658AA/item#dept=m...,JIL SANDER,Shorts & Bermuda,958.0,228.0,36,0.237996
